In [1]:
import pandas as pd
import bokeh
import numpy as np
from bokeh.plotting import figure, show, output_file
from collections import OrderedDict
from bokeh.layouts import layout, widgetbox, column
from bokeh.models import CustomJS, ColumnDataSource, Slider, FactorRange, Whisker
from bokeh.io import output_file, show, export_png, output_notebook, export_svg
from bokeh.plotting import figure
from bokeh.transform import dodge


In [2]:
excel = pd.read_excel('Survey Result.xlsx', sheet_name=None)

/home/clement/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  """Entry point for launching an IPython kernel.


In [3]:
USERS = ['Boucher', 'Renaud Duval', 'Ferreira']

METHODS = ['Attention_ViT', 'DTD_ResNet152', 'LRP_ViT']
COLUMNS =  ['Comments'] + METHODS
cross_data = {}
cross_data[('Images', '')] = excel[USERS[0]]['Images']

for m in COLUMNS:
    for u in USERS:
        
        cross_data[(m,u)] = excel[u][m]


df = pd.DataFrame(cross_data)
df['class'] = df['Images'].apply(lambda x: x.split('-')[0], )

In [4]:
df = df.sort_values('class')

In [5]:
df

Images Comments                            Attention_ViT  \
                            Boucher Renaud Duval      Ferreira       Boucher   
149     CNV-8713483-1.jpeg      NaN          NaN           NaN             1   
100     CNV-9499888-1.jpeg      NaN          NaN           NaN             2   
99      CNV-6725223-1.jpeg      NaN          NaN           NaN             3   
207      CNV-235517-1.jpeg      NaN          NaN           NaN             3   
94      CNV-6900110-2.jpeg      NaN          NaN           NaN             2   
..                     ...      ...          ...           ...           ...   
109  DRUSEN-5097014-2.jpeg      NaN          NaN           NaN             3   
110  DRUSEN-2899654-1.jpeg      NaN          NaN           NaN             3   
195  DRUSEN-7995034-1.jpeg      NaN          NaN           NaN             3   
236  DRUSEN-3273592-2.jpeg      NaN          NaN           NaN             2   
299  DRUSEN-2099732-3.jpeg      NaN          NaN  It is drusen             2   

                          DTD_ResNet152                       LRP_ViT  \
    Renaud Duval Ferreira       Boucher Renaud Duval Ferreira Boucher   
149            3        1             2            2        2       3   
100            1        2             3            3        3       1   
99             2        3             2            3        1       1   
207            3        3             2            2        1       1   
94             2        3             3            3        2       1   
..           ...      ...           ...          ...      ...     ...   
109            3        2             1            2        1       2   
110            3        3             1            2        1       2   
195            3        3             1            1        1       2   
236            3        3             1            1        1       3   
299            2        2             1            1        1       3   

                            class  
    Renaud Duval Ferreira          
149            1        3     CNV  
100            2        1     CNV  
99             1        2     CNV  
207            1        2     CNV  
94             1        1     CNV  
..           ...      ...     ...  
109            1        3  DRUSEN  
110            1        2  DRUSEN  
195            2        2  DRUSEN  
236            2        2  DRUSEN  
299            3        3  DRUSEN  

[300 rows x 14 columns]

In [6]:
average = {}
stds = {}

EXPERTS = ['Expert 1', 'Expert 2', 'Expert 3'] 
average['Experts'] = EXPERTS


x = [ (e, m) for e in EXPERTS for m in METHODS ]

for m in METHODS:
    average[m] = []
    stds[m] = []

    for u in USERS:
         average[m].append(np.mean(excel[u][m]))
         stds[m].append(np.var(excel[u][m]))

means = np.asarray(sum(zip(average[METHODS[0]], average[METHODS[1]], average[METHODS[2]]), ())) # like an hstack
std = np.asarray(sum(zip(stds[METHODS[0]], stds[METHODS[1]], stds[METHODS[2]]), ())) # like an hstack

In [7]:
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral3

source = ColumnDataSource(data=dict(x=x, means=means, lower=means-std/2, upper=means+std/2))

p = figure(x_range=FactorRange(*x), plot_height=350, plot_width=700,
           toolbar_location=None, tools="")

p.y_range.start = 0
p.y_range.end = 3.

p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None



p.vbar(x='x', top='means', width=0.9, source=source, line_color="white",
       # use the palette to colormap based on the the x[1:2] values
       fill_color=factor_cmap('x', palette=Spectral3, factors=METHODS, start=1, end=2))


w = Whisker(source=source, base="x", upper="upper", lower="lower", level='overlay')
p.add_layout(w)
w.upper_head.line_alpha = 0.5
w.lower_head.line_alpha = 0.5
p.xaxis.separator_line_width = 1
p.xaxis.major_label_text_font_size = "15px"
p.xaxis.group_text_font_size = "15px"
p.yaxis.major_label_text_font_size = "15px"
output_notebook()
show(p)

p.background_fill_color = None
p.border_fill_color = None

export_png(p, filename='per_expert_method_ranking.png')
p.output_backend = "svg"
export_svg(p, filename='per_expert_method_ranking.svg')

Loading BokehJS ...

['per_expert_method_ranking.svg']

In [8]:
df[('Attention_ViT', 'Avg_score')] = df['Attention_ViT'].mean(axis=1)
df[('DTD_ResNet152', 'Avg_score')] = df['DTD_ResNet152'].mean(axis=1)
df[('LRP_ViT', 'Avg_score')] = df['LRP_ViT'].mean(axis=1)

In [9]:
CLASSES = ['DRUSEN', 'CNV', 'DME']
x = []
avgs = []
stds = []

for c in CLASSES:
    d = df.loc[df['class']==c]
#     for e, u in zip(EXPERTS, USERS):
    for m in METHODS:
        x.append((c, m))
        avgs.append(d[(m, 'Avg_score')].mean())
        stds.append(d[(m, 'Avg_score')].std())

avgs = np.asarray(avgs)
stds = np.asarray(stds)

In [10]:

source = ColumnDataSource(data=dict(x=x, means=avgs, upper=avgs+stds/2, lower=avgs-stds/2))

p = figure(x_range=FactorRange(*x), plot_height=350, plot_width=700,
           toolbar_location=None, tools="")

p.y_range.start = 0
p.y_range.end = 3.

p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None


p.add_layout(
    Whisker(source=source, base="x", upper="upper", lower="lower", dimension='height', 
            level='overlay')
)


p.vbar(x='x', top='means', width=0.9, source=source, line_color="white",
       # use the palette to colormap based on the the x[1:2] values
       fill_color=factor_cmap('x', palette=Spectral3, factors=METHODS, start=1, end=2))

p.xaxis.separator_line_width = 1
p.xaxis.major_label_text_font_size = "15px"
p.xaxis.group_text_font_size = "15px"
p.yaxis.major_label_text_font_size = "15px"



p.xaxis.separator_line_width = 2
output_notebook()
show(p)
export_png(p, filename='per_class_method_ranking.png')
p.output_backend = "svg"
export_svg(p, filename='per_class_method_ranking.svg')

Loading BokehJS ...

['per_class_method_ranking.svg']

In [11]:
from scipy.stats import friedmanchisquare

In [18]:
survey_per_user = {}
for u in USERS:
    ranks = []
    for m in METHODS:
        ranks.append(df[m, u].to_numpy())
    
    survey_per_user[u] = np.asarray(ranks).transpose()

sum_ranks = np.zeros_like(survey_per_user[u])
for k, v in survey_per_user.items():
        sum_ranks += v

In [23]:
friedmanchisquare(*(survey_per_user[USERS[2]]).transpose())

friedmanchisquare(*sum_ranks.transpose())

FriedmanchisquareResult(statistic=62.768054375531065, pvalue=2.344715597480587e-14)

In [24]:
import scikit_posthocs as sp
sp.posthoc_conover_friedman(sum_ranks)

,0,1,2
0,1.000000e+00,0.000287,1.206504e-14
1,2.870947e-04,1.000000,2.307145e-05
2,1.206504e-14,0.000023,1.000000e+00


In [16]:
p_values = None
for u in USERS:
    if p_values is None:
        p_values = -np.log(sp.posthoc_conover_friedman(survey_per_user[u]))
    else:
        p_values = p_values + sp.posthoc_conover_friedman(survey_per_user[u])

In [17]:
p_values

,0,1,2
0,3.000000e+00,0.014821,4.850442e-11
1,1.482092e-02,3.000000,2.919410e-02
2,4.850442e-11,0.029194,3.000000e+00


In [25]:
METHODS

['Attention_ViT', 'DTD_ResNet152', 'LRP_ViT']